
# Batch JSON → Interactive 3D HTML

**Expected JSON schema** :
```json
{
  "lines": [
    { "x":[...], "y":[...], "z":[...], "color":[r,g,b] or "#RRGGBB", "label":"..." },
    ...
  ],
  "title": "optional figure title"
}
```

In [29]:

from __future__ import annotations
import json, re
from pathlib import Path
from typing import List, Sequence, Tuple, Union

import numpy as np
import plotly.graph_objects as go
import plotly.express as px

ColorLike = Union[str, List[float], Tuple[float, float, float]]

HEX_RE = re.compile(r'^#([0-9A-Fa-f]{6})$')
RGB_TXT_RE = re.compile(r'^\s*rgb\(\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*\)\s*$')
CSV_RE = re.compile(r'^\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*$')


def to_hex_from_triplet(tri: Sequence[float]) -> str:
    """Convert a numeric RGB triplet to '#RRGGBB'. Accepts [0..1] or [0..255]."""
    if len(tri) != 3:
        raise ValueError("RGB triplet must have length 3.")
    tri = list(map(float, tri))
    if any(v > 1.0 for v in tri):
        r, g, b = [int(round(max(0, min(255, v)))) for v in tri]
    else:
        r, g, b = [int(round(max(0, min(1.0, v)) * 255)) for v in tri]
    return "#{:02x}{:02x}{:02x}".format(r, g, b)


def normalize_color(c: ColorLike | None, fallback: str) -> str:
    """Normalize various color formats to '#RRGGBB'."""
    if c is None:
        return fallback
    if isinstance(c, (list, tuple)):
        return to_hex_from_triplet(c)
    if isinstance(c, str):
        s = c.strip()
        if HEX_RE.match(s):
            return s
        m = RGB_TXT_RE.match(s) or CSV_RE.match(s)
        if m:
            tri = [float(m.group(1)), float(m.group(2)), float(m.group(3))]
            return to_hex_from_triplet(tri)
    return fallback


def validate_lines(x_lines: Sequence[Sequence[float]],
                   y_lines: Sequence[Sequence[float]],
                   z_lines: Sequence[Sequence[float]]):
    if not (len(x_lines) == len(y_lines) == len(z_lines)):
        raise ValueError("Mismatched number of lines among x/y/z.")
    for i, (x, y, z) in enumerate(zip(x_lines, y_lines, z_lines)):
        if not (len(x) == len(y) == len(z)):
            raise ValueError(f"Line {i}: x/y/z length mismatch ({len(x)}/{len(y)}/{len(z)}).")
        if len(x) == 0:
            raise ValueError(f"Line {i} is empty.")


In [30]:

def load_lines_from_json(json_path: Path):
    """Load x/y/z + color + label + optional title from JSON."""
    with json_path.open("r", encoding="utf-8") as f:
        obj = json.load(f)

    x_lines, y_lines, z_lines = [], [], []
    colors, labels = [], []

    if "lines" in obj and isinstance(obj["lines"], list):
        for i, item in enumerate(obj["lines"]):
            x = list(map(float, item["x"]))
            y = list(map(float, item["y"]))
            z = list(map(float, item["z"]))
            x_lines.append(x); y_lines.append(y); z_lines.append(z)

            colors.append(item.get("color"))
            labels.append(item.get("label", f"Line {i+1}"))
        title = obj.get("title")
    else:
        # Backward compatible schema: x_lines/y_lines/z_lines only
        x_lines = [list(map(float, arr)) for arr in obj["x_lines"]]
        y_lines = [list(map(float, arr)) for arr in obj["y_lines"]]
        z_lines = [list(map(float, arr)) for arr in obj["z_lines"]]
        colors = [None] * len(x_lines)
        labels = [f"Line {i+1}" for i in range(len(x_lines))]
        title = obj.get("title")

    validate_lines(x_lines, y_lines, z_lines)
    return x_lines, y_lines, z_lines, colors, labels, title


In [31]:

def plot3d_lines_to_html(x_lines, y_lines, z_lines, colors_in, labels, out_html: Path,
                         marker_count: int = 25, marker_size: int = 2, title: str | None = None):
    out_html.parent.mkdir(parents=True, exist_ok=True)

    palette = px.colors.qualitative.Plotly  # fallback palette
    n_colors = len(palette)

    fig = go.Figure()
    for i, (x, y, z) in enumerate(zip(x_lines, y_lines, z_lines)):
        x = np.asarray(x, dtype=float)
        y = np.asarray(y, dtype=float)
        z = np.asarray(z, dtype=float)
        T = x.shape[0]

        fallback = palette[i % n_colors]
        color_hex = normalize_color(colors_in[i], fallback)
        name = labels[i] if labels and i < len(labels) else f"Line {i+1}"

        # Line
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode="lines",
            line=dict(width=4, color=color_hex),
            name=name
        ))

        # Sampled markers
        mk = np.unique(np.round(np.linspace(0, T - 1, int(min(T, max(marker_count, 1)))))).astype(int)
        fig.add_trace(go.Scatter3d(
            x=x[mk], y=y[mk], z=z[mk],
            mode="markers",
            marker=dict(size=marker_size, color="white", line=dict(color=color_hex, width=1)),
            name=f"{name} marks",
            showlegend=False
        ))

        # Start (red) & Mid (Purple) & End (black)
        fig.add_trace(go.Scatter3d(x=[x[0]], y=[y[0]], z=[z[0]],
                                   mode="markers",
                                   marker=dict(size=marker_size, color="red"),
                                   name=f"{name} start", showlegend=False))
        mid = len(x) // 2
        fig.add_trace(go.Scatter3d(
            x=[x[mid]], y=[y[mid]], z=[z[mid]],
            mode="markers",
            marker=dict(size=marker_size, color="#FF00FF"),
            name=f"{name} middle", showlegend=False
        ))
        fig.add_trace(go.Scatter3d(x=[x[-1]], y=[y[-1]], z=[z[-1]],
                                   mode="markers",
                                   marker=dict(size=marker_size, color="black"),
                                   name=f"{name} end", showlegend=False))

    fig.update_layout(
        title=title or "Interactive 3D Trajectories",
        scene=dict(xaxis_title="PC1", yaxis_title="PC2", zaxis_title="PC3"),
        template="plotly_white",
        margin=dict(l=0, r=0, t=48, b=0),
        legend=dict(itemsizing="trace")
    )

    fig.write_html(str(out_html), include_plotlyjs="cdn", full_html=True)


In [32]:

def batch_convert(src_dir: Path, dst_dir: Path, marker_count: int = 25):
    """Recursively convert all .json under src_dir into .html under dst_dir, preserving structure."""
    if not src_dir.exists():
        raise FileNotFoundError(f"Source directory not found: {src_dir}")
    json_files = sorted(src_dir.rglob("*.json"))
    if not json_files:
        print(f"⚠️ No .json files found in {src_dir}")
        return

    print(f"🔎 Found {len(json_files)} JSON file(s). Output root: {dst_dir}")
    ok = fail = 0
    for jp in json_files:
        rel = jp.relative_to(src_dir)
        out_html = dst_dir / rel.with_suffix(".html")
        try:
            x_lines, y_lines, z_lines, colors, labels, title = load_lines_from_json(jp)
            plot3d_lines_to_html(
                x_lines, y_lines, z_lines, colors, labels, out_html,
                marker_count=marker_count, title=title
            )
            print(f"✅ {rel}  ->  {out_html.relative_to(dst_dir)}")
            ok += 1
        except Exception as e:
            print(f"❌ {rel} failed: {e}")
            fail += 1

    print(f"\nDone. Success: {ok}, Failed: {fail}. Output: {dst_dir.resolve()}")

## Run batch conversion

In [37]:

# Configure your paths here (relative to the notebook or absolute)
src = Path('../results/json_temp/pca/second_shift/')
dst = Path('../results/html_final/pca/second_shift/')

# Change marker_count if needed
batch_convert(src, dst, marker_count=25)

🔎 Found 10 JSON file(s). Output root: ..\results\html_final\pca\second_shift
✅ RB03-PFC\alignLick\correct_all_rules.json  ->  RB03-PFC\alignLick\correct_all_rules.html
✅ RB03-PFC\alignLick\correct_false_all_rules.json  ->  RB03-PFC\alignLick\correct_false_all_rules.html
✅ RB03-PFC\alignLick\correct_miss_all_rules.json  ->  RB03-PFC\alignLick\correct_miss_all_rules.html
✅ RB03-PFC\alignLick\false_all_rules.json  ->  RB03-PFC\alignLick\false_all_rules.html
✅ RB03-PFC\alignLick\miss_all_rules.json  ->  RB03-PFC\alignLick\miss_all_rules.html
✅ RB03-PFC\alignTrack\correct_all_rules.json  ->  RB03-PFC\alignTrack\correct_all_rules.html
✅ RB03-PFC\alignTrack\correct_false_all_rules.json  ->  RB03-PFC\alignTrack\correct_false_all_rules.html
✅ RB03-PFC\alignTrack\correct_miss_all_rules.json  ->  RB03-PFC\alignTrack\correct_miss_all_rules.html
✅ RB03-PFC\alignTrack\false_all_rules.json  ->  RB03-PFC\alignTrack\false_all_rules.html
✅ RB03-PFC\alignTrack\miss_all_rules.json  ->  RB03-PFC\alignTrack


## (Optional) Convert a single JSON file

Uncomment and point to a single JSON to convert it and inspect output quickly.


In [34]:

# Example:
# single_json = Path('results/json_temp/demo/lines.json')
# out_html = Path('results/html_final/demo/lines.html')
# x_lines, y_lines, z_lines, colors, labels, title = load_lines_from_json(single_json)
# plot3d_lines_to_html(x_lines, y_lines, z_lines, colors, labels, out_html, marker_count=25, title=title)
# out_html


## (Optional) Generate dummy JSON for testing

This cell generates a small demo JSON compatible with the expected schema, so you can test end-to-end without MATLAB.


In [35]:

# import os, json
# os.makedirs('results/json_temp/demo', exist_ok=True)
# demo_json = 'results/json_temp/demo/lines.json'

# t = np.linspace(0, 2*np.pi, 200)
# x1, y1, z1 = np.cos(t), np.sin(t), t*0.1
# x2, y2, z2 = np.cos(t)*0.5, np.sin(t)*0.5, t*0.15

# data = {
#     "title": "Demo Trajectories",
#     "lines": [
#         {"x": x1.tolist(), "y": y1.tolist(), "z": z1.tolist(), "color": [0.2, 0.6, 1.0], "label": "Group A"},
#         {"x": x2.tolist(), "y": y2.tolist(), "z": z2.tolist(), "color": "#ff5500", "label": "Group B"}
#     ]
# }
# with open(demo_json, "w", encoding="utf-8") as f:
#     json.dump(data, f, ensure_ascii=False)
# print(f"✅ Wrote demo JSON: {demo_json}")